In [45]:
import warnings
warnings.filterwarnings('ignore')

In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score, confusion_matrix
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder

In [47]:
data = pd.read_csv(r'C:\Users\sylvia.pereira\OneDrive - alteryx.com\Desktop\Ryerson\CIND_820 XJH\Dataset_BBB_Association_Analysis_Final.csv')
data.head()

,id_student,activity_type,sum_click,final_result
0,275296,forumng,13,Fail
1,275296,glossary,0,Fail
2,275296,oucollaborate,0,Fail
3,275296,oucontent,60,Fail
4,275296,ouelluminate,0,Fail


In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59037 entries, 0 to 59036
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_student     59037 non-null  int64 
 1   activity_type  59037 non-null  object
 2   sum_click      59037 non-null  int64 
 3   final_result   59037 non-null  object
dtypes: int64(2), object(2)
memory usage: 1.8+ MB


In [49]:
# Clean the data
data.dropna(axis=0, subset=['id_student'], inplace=True)
data['id_student'] = data['id_student'].astype('str')
data = data[~data['id_student'].str.contains('C')]

# Create a one-hot encoded matrix of the transactions
basket = (data.groupby(['id_student', 'activity_type'])['sum_click']
          .sum().unstack().reset_index().fillna(0)
          .set_index('id_student'))

# Convert the transaction matrix into binary values
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
basket_sets = basket.applymap(encode_units)

# Generate frequent itemsets using Apriori algorithm
frequent_itemsets = apriori(basket_sets, min_support=0.4, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Show the top 5 rules by lift
rules.nlargest(5, 'lift')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
564,"(resource, subpage, quiz)","(url, oucontent, forumng)",0.884802,0.711538,0.679238,0.767673,1.078891,0.049668,1.241616,0.634754
577,"(url, oucontent, forumng)","(resource, subpage, quiz)",0.711538,0.884802,0.679238,0.954605,1.078891,0.049668,2.537683,0.253491
552,"(url, subpage, oucontent, forumng)","(resource, quiz)",0.711165,0.885922,0.679238,0.955106,1.078093,0.049201,2.541063,0.250787
589,"(resource, quiz)","(url, subpage, oucontent, forumng)",0.885922,0.711165,0.679238,0.766702,1.078093,0.049201,1.238050,0.634970
408,"(resource, quiz)","(url, oucontent, forumng)",0.885922,0.711538,0.679238,0.766702,1.077527,0.048870,1.236450,0.630701


In [50]:
# Clean the data
data.dropna(axis=0, subset=['id_student'], inplace=True)
data['id_student'] = data['id_student'].astype('str')
data = data[~data['id_student'].str.contains('C')]

# Create a one-hot encoded matrix of the transactions
basket = (data.groupby(['id_student', 'activity_type'])['sum_click']
          .sum().unstack().reset_index().fillna(0)
          .set_index('id_student'))

# Convert the transaction matrix into binary values
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
basket_sets = basket.applymap(encode_units)

# Generate maximal itemsets using Apriori algorithm
maximal_itemsets = apriori(basket_sets, max_len=None, use_colnames=True, verbose=0)

# Display the top 10 maximal itemsets
maximal_itemsets.nlargest(10, 'support')

,support,itemsets
4,0.981143,(subpage)
3,0.968073,(resource)
18,0.963779,"(resource, subpage)"
0,0.911314,(forumng)
9,0.901979,"(subpage, forumng)"
2,0.900299,(quiz)
16,0.894884,"(subpage, quiz)"
8,0.892644,"(resource, forumng)"
28,0.890403,"(resource, subpage, forumng)"
15,0.885922,"(resource, quiz)"
